# Imports

In [42]:
import torch
import torchvision
from torchvision.transforms import functional as F
import cv2
import numpy as np
from tqdm import tqdm
import os

In [44]:
# Ruta del video de entrada  y salida

In [46]:
video_path = r"D:\Lorena\Downloads\resultado_furbo_short (1).mp4"
output_path = r"D:\Lorena\Downloads\resultado_fasterrcnn_persons.mp4"

#  Cargar modelo Faster R-CNN preentrenado

In [49]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

# Usar GPU si está disponible


In [52]:
# Usar GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [54]:
# Clases COCO (solo usaremos 'person', índice 1)

In [56]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus'
]

# Abrir video

In [65]:
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Inicializar video de salida


In [75]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print("Procesando vídeo...")

for _ in tqdm(range(total_frames)):
    ret, frame = cap.read()
    if not ret:
        break

    img_tensor = F.to_tensor(frame).to(device)

    with torch.no_grad():
        predictions = model([img_tensor])[0]

    for box, label, score in zip(predictions['boxes'], predictions['labels'], predictions['scores']):
        if score > 0.6 and label.item() == 1:  # En COCO, la clase 'person' tiene ID 1
            x1, y1, x2, y2 = box.int().tolist()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'person {score:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    out.write(frame)

cap.release()
out.release()
print(f" Video guardado en: {output_path}")


Procesando vídeo...


100%|█████████▉| 423/424 [16:15<00:02,  2.31s/it]

✅ Video guardado en: D:\Lorena\Downloads\resultado_fasterrcnn_persons.mp4


In [1]:
import torch
import torchvision
from torchvision.transforms import functional as F
import cv2
import numpy as np
from tqdm import tqdm
import os
import time  # Para medir tiempos

# Rutas
video_path = r"D:\Lorena\Downloads\resultado_furbo_short (1).mp4"
output_path = r"D:\Lorena\Downloads\resultado_fasterrcnn_persons.mp4"

# Modelo
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

print("Procesando vídeo...")

# Métricas
total_persons = 0
frame_count = 0
pre_times = []
inf_times = []
post_times = []

# Bucle por frame
for _ in tqdm(range(total_frames)):
    ret, frame = cap.read()
    if not ret:
        break

    start_pre = time.time()
    img_tensor = F.to_tensor(frame).to(device)
    pre_times.append(time.time() - start_pre)

    start_inf = time.time()
    with torch.no_grad():
        predictions = model([img_tensor])[0]
    inf_times.append(time.time() - start_inf)

    start_post = time.time()
    person_count = 0
    for box, label, score in zip(predictions['boxes'], predictions['labels'], predictions['scores']):
        if score > 0.6 and label.item() == 1:
            x1, y1, x2, y2 = box.int().tolist()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'person {score:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
            person_count += 1
    post_times.append(time.time() - start_post)

    total_persons += person_count
    frame_count += 1
    out.write(frame)

cap.release()
out.release()

# Resultados
print(f"\n✅ Video guardado en: {output_path}")
print("\n📊 Métricas de Faster R-CNN:")
print(f"Media Personas detectadas: {total_persons / frame_count:.2f}")
print(f"Velocidad Preprocesamiento (ms): {1000 * sum(pre_times)/frame_count:.2f}")
print(f"Inferencia (ms): {1000 * sum(inf_times)/frame_count:.2f}")
print(f"Postprocesamiento por imagen (ms): {1000 * sum(post_times)/frame_count:.2f}")



C:\Users\Lorena\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Lorena\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Procesando vídeo...


100%|██████████| 424/424 [18:06<00:00,  2.56s/it]


✅ Video guardado en: D:\Lorena\Downloads\resultado_fasterrcnn_persons.mp4

📊 Métricas de Faster R-CNN:
Media Personas detectadas: 5.33
Velocidad Preprocesamiento (ms): 7.50
Inferencia (ms): 2531.36
Postprocesamiento por imagen (ms): 1.00
